In [ ]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-20 20:53:50,527 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-20 20:53:50,529 - settings - Setting database.user to celiib
INFO - 2021-02-20 20:53:50,530 - settings - Setting database.password to newceliipass
INFO - 2021-02-20 20:53:50,535 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-20 20:53:50,535 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-20 20:53:50,549 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-20 20:53:50,891 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-20 20:53:50,924 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-20 20:53:51,283 - settings - Setting enable_python_native_blobs to True


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su
from importlib import reload
import trimesh_utils as tu

# Getting a more high fidelity axon skeleton

In [6]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [7]:
curr_index = 3010
segment_id = multi_axon_segs[curr_index]
split_index = multi_axon_splits[curr_index]
segment_id,split_index

(864691135539149252, 1)

In [18]:
segment_id,split_index = 864691135426651890,0

In [19]:
minnie.AutoProofreadNeurons2() & dict(segment_id = segment_id,
                                      split_index=split_index)

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691135426651890,0,0.25,30.00,0,0,0,1,1,excitatory,densely_spined,2,2,1135.6141158087469,1314.1385539441028,334179,1113.7,1,1,197203,187208,1517

In [20]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

Synapsse Results:
Total Number of Synapses = 129
n_valid_syn_centers_presyn = 28
n_errored_syn_centers_presyn = 15
n_valid_syn_centers_postsyn = 86
n_errored_syn_centers_postsyn = 0
Including axon mesh


In [21]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1


In [22]:
if type(neuron_obj) == tuple:
    neuron_obj = neuron_obj[0][split_index]

In [23]:
neuron_obj.segment_id

864691135426651890

In [24]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [25]:
clu.axon_classification(neuron_obj,
                        plot_candidates=True,
                       best_axon=True,
                       plot_axons=True)


Part 1: Axon like branchese 
{'L0': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37]), 'L2': array([0]), 'L5': array([1]), 'L7': array([0]), 'L8': array([0])}
Limb L0 soma angle: 164.26 
Limb L1 soma angle: 106.84 
Limb L2 soma angle: 17.13 
Limb L3 soma angle: 49.37 
Limb L4 soma angle: 136.14 
Limb L5 soma angle: 160.68 
Limb L6 soma angle: 85.6 
Limb L7 soma angle: 81.61 
Limb L8 soma angle: 95.44 

Part 2: possible_axon_limbs = [0, 1, 4, 5, 6, 7, 8]

Phase 2: Working on Limb L0
nodes_to_eliminate = []

Phase 2: Working on Limb L1
nodes_to_eliminate = [6]

Phase 2: Working on Limb L4
nodes_to_eliminate = [0]

Phase 2: Working on Limb L5
nodes_to_eliminate = [2]

Phase 2: Working on Limb L6
nodes_to_eliminate = []

Phase 2: Working on Limb L7
nodes_to_eliminate = [0]
The filtered network is empty so just leaving the candidates as empty lists

Phase 2: Working o

Final Candidates

Limb 0
Candidate 0 (blue): array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37])

Limb 1
Candidate 0 (magenta): array([4])
Candidate 1 (tan): array([0, 1, 2, 3, 5])

Limb 4
Candidate 0 (orange): array([1])
Candidate 1 (cyan): array([2, 3, 4])

Limb 5
Candidate 0 (gray): array([0])
Candidate 1 (darkslategray): array([1])

Limb 6
Candidate 0 (blue): array([0, 1, 2])

Limb 7
Candidate 0 (brown): array([0])

Limb 8
Candidate 0 (olive): array([0])
Part 3: axon_subgraph_candidates = {0: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]], 1: [[4], [0, 1, 2, 3, 5]], 4: [[1], [2, 3, 4]], 5: [[0], [1]], 6: [[0, 1, 2]], 7: [[0]], 8: [[0]]}


 --- Working on limb 0, candidate # 0
38 out of 38 branches are axons
Axon percentage = 1.0
curr_axon_angle_threshold = 95
candid

({'L0': array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
         13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
         26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37.])},
 {})

# Should not be absolutely tiny at the start

In [36]:
neuron_obj.preprocessed_data["limb_network_stating_info"][0]

{0: {0: {'touching_verts': TrackedArray([[789305.9, 756181.4, 608356.1],
                 [790240.5, 756210. , 608044.5],
                 [789318.4, 756236.1, 608083.2],
                 [790146. , 756252. , 607908. ],
                 [789726. , 755895. , 608412. ],
                 [789978. , 755790. , 608433. ],
                 [789735. , 756183.9, 607682.6],
                 [789348. , 756294. , 607971. ],
                 [789339.5, 756278.3, 608261. ],
                 [789432. , 756052.5, 608485.5],
                 [789400.5, 756304.5, 607871.2],
                 [789531.4, 756218.2, 607709.7],
                 [790144.5, 755929.4, 608369.4],
                 [789907.9, 756191.2, 607755.3],
                 [789872.9, 755893.8, 608452.2],
                 [789639.6, 756372.7, 607791.6],
                 [789558. , 756084. , 608412. ],
                 [790158.6, 756062.9, 608195. ],
                 [790192.9, 756104.8, 608006.9],
                 [789703. , 756291. , 607756.

In [41]:
new_limb_corr, st_info = pre.high_fidelity_axon_decomposition(neuron_obj,
                                     plot_new_axon_limb_correspondence=False,
                                     plot_connecting_skeleton_fix = False,
                                     return_starting_info=True,
                                verbose = True,
)

Starting axon branch was the starting node so using border vertices and NOT computing upstream branch
upstream_node_to_axon_starting_branch = None


  0%|          | 0/1 [00:00<?, ?it/s]

soma_touching_vertices_dict = {0: [TrackedArray([[789305.9, 756181.4, 608356.1],
              [790240.5, 756210. , 608044.5],
              [789318.4, 756236.1, 608083.2],
              [790146. , 756252. , 607908. ],
              [789726. , 755895. , 608412. ],
              [789978. , 755790. , 608433. ],
              [789735. , 756183.9, 607682.6],
              [789348. , 756294. , 607971. ],
              [789339.5, 756278.3, 608261. ],
              [789432. , 756052.5, 608485.5],
              [789400.5, 756304.5, 607871.2],
              [789531.4, 756218.2, 607709.7],
              [790144.5, 755929.4, 608369.4],
              [789907.9, 756191.2, 607755.3],
              [789872.9, 755893.8, 608452.2],
              [789639.6, 756372.7, 607791.6],
              [789558. , 756084. , 608412. ],
              [790158.6, 756062.9, 608195. ],
              [790192.9, 756104.8, 608006.9],
              [789703. , 756291. , 607756.6],
              [790046.2, 756220.5, 607855.5],

100%|██████████| 1/1 [00:02<00:00,  2.72s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 3.3397390842437744
branches_touching_root = [22]
combining close nodes
combine_threshold = 1300
length of Graph = 14260
Working on path [555. 557. 559. 560.]
path_degrees = [4, 2, 2, 3]
Working on path [2908. 2915. 2923. 2932. 2950. 2969. 2973.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [4282. 4289. 4296. 4301. 4306. 4313. 4317. 4321. 4325. 4328. 4331. 4332.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
Working on path [5902. 5926. 5949. 5953.]
path_degrees = [3, 2, 2, 3]
Working on path [6011. 6029. 6039.]
path_degrees = [3, 2, 3]
Working on path [9289. 9293. 9299. 9304. 9309. 9310.]
path_degrees = [3, 2, 2, 2, 2, 3]
Working on path [12884. 12907. 12920. 12933. 12945. 12960. 12970. 12980. 12984.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 2, 3]
Working on path [12968. 12981. 12995.]
path_degrees = [3, 2, 3]
Working on path [14266. 12988. 12987. 12986.]
path_degrees = [4, 2, 2, 3]
Working on path [14268. 129

...


Decomposing first pass: 11.022558450698853
Only Using MeshParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 2.765655517578125e-05
Total time for MAP sublimb processing 4.291534423828125e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 0.00010824203491210938
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.0014483928680419922
sbv[0].reshape(-1,3) = [[789305.9 756181.4 608356.1]]
closest_sk_pt_coord BEFORE = [789936.85782423 756386.20058738 607863.09383671]
current_skeleton.shape = (10, 2, 3)
node_for_stitch = 0: [789936.85782423 756386.20058738 607863.09383671]
node_for_stitch AFTER = 0: [789936.85782423 756386.20058738 607863.09383671]
possible_node_loc = [0 4]
possible_node_loc AFTER = [0 4]
curr_shortest_path = [0]
Current stitch point was a branch or endpoint
change_status for create soma ext


Removed 55 skeletal branches



113368 113369
For marked faces: None
empty_indices % = 0.00216990535331528
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test



Upstream node was None so don't have to adjust


# The whole process of replacing the axon with higher fidelity

nucleus_id
1841
1896
2005
2020
2242
2260
2329
2417
2433
2765


In [46]:
"""
Purpose: To replace the axon branches with a higher
fidelity representation within the neuron object
(aka replacing all of the branch objects)

Pseudocode:
0) Get the limb branch dict for the axon (if empty then return)
1) Generate the new limb correspondence for the axon
2) Combine the data with any left over branches that still exist 
in the limb object

a. Figure out if the starting branch was in the axon group or not
--> if in axon group then use starting limb informatoin from there
(if not then use existting)

b. Delete all replaced branches

c. Rename the existing branches so not incorporate any of the new names

d. 

"""

